In [3]:
import pandas as pd

# Load the data
decision_tree = pd.read_csv('decision_trees.csv')
gradient_boosting = pd.read_csv('gradient_boost.csv')
logistic_regression = pd.read_csv('logistic_regression.csv')
random_forest = pd.read_csv('random_forest.csv')

# Display the first few rows of each dataset to understand their structure
print("Decision Tree Predictions:")
print(decision_tree.head())
print("\nGradient Boosting Predictions:")
print(gradient_boosting.head())
print("\nLogistic Regression Predictions:")
print(logistic_regression.head())
print("\nRandom Forest Predictions:")
print(random_forest.head())


Decision Tree Predictions:
         TrackID  Predictor
0  199810_208019          0
1    199810_9903          0
2  199810_242681          0
3   199810_74139          1
4   199810_18515          1

Gradient Boosting Predictions:
         TrackID  Predictor
0  199810_208019          0
1    199810_9903          0
2  199810_242681          0
3   199810_74139          1
4   199810_18515          1

Logistic Regression Predictions:
         TrackID  Predictor
0  199810_208019          0
1    199810_9903          0
2  199810_242681          0
3   199810_74139          1
4   199810_18515          1

Random Forest Predictions:
         TrackID  Predictor
0  199810_208019          0
1   199810_74139          1
2    199810_9903          0
3  199810_242681          0
4   199810_18515          0


In [4]:
# Merging the datasets on TrackID
merged_data = decision_tree.merge(gradient_boosting, on='TrackID', suffixes=('_dt', '_gb'))
merged_data = merged_data.merge(logistic_regression, on='TrackID')
merged_data = merged_data.merge(random_forest, on='TrackID', suffixes=('_lr', '_rf'))

# Renaming columns for clarity
merged_data.columns = ['TrackID', 'Decision_Tree', 'Gradient_Boosting', 'Logistic_Regression', 'Random_Forest']

# Display the merged data
print(merged_data.head())


         TrackID  Decision_Tree  Gradient_Boosting  Logistic_Regression  \
0  199810_208019              0                  0                    0   
1    199810_9903              0                  0                    0   
2  199810_242681              0                  0                    0   
3   199810_74139              1                  1                    1   
4   199810_18515              1                  1                    1   

   Random_Forest  
0              0  
1              0  
2              0  
3              1  
4              0  


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Prepare the data
features = merged_data[['Decision_Tree', 'Gradient_Boosting', 'Logistic_Regression', 'Random_Forest']]
labels = merged_data['Decision_Tree']  # Assuming 'Decision_Tree' column as the label for illustration

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Train the logistic regression meta-model
meta_model = LogisticRegression()
meta_model.fit(X_train, y_train)

# Predict and evaluate the model
predictions = meta_model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

# Display the accuracy
accuracy


1.0

In [9]:
# Setup a quick meta-model using Logistic Regression without cross-validation due to time constraints
meta_model_quick = LogisticRegression()
meta_model_quick.fit(features, labels)
merged_data['Meta_Predictor'] = meta_model_quick.predict(features)

# Save to CSV
csv_path = 'ensemble_predictions.csv'
merged_data.to_csv(csv_path, index=False)
csv_path


'ensemble_predictions.csv'

In [10]:
# Using fewer estimators for demonstration
simple_estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('dt', DecisionTreeClassifier(max_depth=1, random_state=42))
]
stacking_model_simple = StackingClassifier(estimators=simple_estimators, final_estimator=LogisticRegression(), cv=3)

# Fit the model
stacking_model_simple.fit(features, labels)

# Generate final predictions
final_predictions = stacking_model_simple.predict(features)

# Creating a DataFrame for the final predictions
final_predictions_df = pd.DataFrame({'TrackID': merged_data['TrackID'], 'Final_Predictor': final_predictions})
final_predictions_df.to_csv('final_ensemble_predictions.csv', index=False)
final_predictions_df.head()


TrackID  Final_Predictor
0  199810_208019                0
1    199810_9903                0
2  199810_242681                0
3   199810_74139                1
4   199810_18515                1

In [11]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Assuming features and labels are already defined

# Initialize classifiers
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = DecisionTreeClassifier(random_state=1)

# Create Voting Classifier
eclf_hard = VotingClassifier(estimators=[
     ('lr', clf1), ('rf', clf2), ('dt', clf3)],
     voting='hard')

eclf_soft = VotingClassifier(estimators=[
     ('lr', clf1), ('rf', clf2), ('dt', clf3)],
     voting='soft', weights=[2,1,1])

# Train the classifiers
eclf_hard.fit(features, labels)
eclf_soft.fit(features, labels)

# Evaluate using cross-validation
from sklearn.model_selection import cross_val_score
scores_hard = cross_val_score(eclf_hard, features, labels, cv=5)
scores_soft = cross_val_score(eclf_soft, features, labels, cv=5)

# Display scores
scores_hard_mean = scores_hard.mean()
scores_soft_mean = scores_soft.mean()
scores_hard_mean, scores_soft_mean


(1.0, 1.0)

In [15]:
# Generate predictions
hard_predictions = eclf_hard.predict(features)
soft_predictions = eclf_soft.predict_proba(features)[:, 1]  # Get the probability for the positive class

# Create a DataFrame with the predictions
results_df = pd.DataFrame({
    'TrackID': merged_data['TrackID'],
    'Hard_Voting_Predictions': hard_predictions,
    'Soft_Voting_Probabilities': soft_predictions
})

# Save to CSV
results_df.to_csv('voting_ensemble_predictions.csv', index=False)


In [16]:
# Hard Voting Predictions
hard_voting_df = pd.DataFrame({
    'TrackID': merged_data['TrackID'],
    'Hard_Voting_Predictions': hard_predictions
})
hard_voting_df.to_csv('hard_voting_predictions.csv', index=False)

# Soft Voting Probabilities
soft_voting_df = pd.DataFrame({
    'TrackID': merged_data['TrackID'],
    'Soft_Voting_Probabilities': soft_predictions
})
soft_voting_df.to_csv('soft_voting_predictions.csv', index=False)

# Stacking Predictions
stacking_predictions_df = pd.DataFrame({
    'TrackID': merged_data['TrackID'],
    'Stacking_Predictions': final_predictions
})
stacking_predictions_df.to_csv('stacking_predictions.csv', index=False)


In [ ]:

from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier

# Setting up the stack ensemble
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('lr', LogisticRegression()),
    ('gb', GradientBoostingClassifier())
]
stack_model = StackingClassifier(estimators=estimators, final_estimator=XGBClassifier(), cv=5)

# Training the stack model
stack_model.fit(X_train, y_train)

# Evaluating the stack model
stack_preds = stack_model.predict(X_test)
stack_accuracy = accuracy_score(y_test, stack_preds)
print('Stack Model Accuracy:', stack_accuracy)


In [ ]:

from sklearn.model_selection import GridSearchCV

# Parameters grid
params = {'weights': [[1,1,1], [2,1,1], [1,2,1], [1,1,2]]}

# Grid search for soft voting classifier
grid = GridSearchCV(estimator=eclf_soft, param_grid=params, cv=5, scoring='accuracy')
grid.fit(features, labels)

# Best parameters and best score
print("Best parameters:", grid.best_params_)
print("Best cross-validation score:", grid.best_score_)


In [ ]:

from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Classification report
print(classification_report(y_test, stack_preds))

# ROC AUC score
roc_auc = roc_auc_score(y_test, stack_model.predict_proba(X_test)[:, 1])
print('ROC AUC Score:', roc_auc)

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, stack_model.predict_proba(X_test)[:, 1])
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()
